In [1]:
import numpy as np
import pandas as pd
import scanpy as sc

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()

from matplotlib import rcParams

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('default')



scanpy==1.4.4.post1 anndata==0.6.22.post1 umap==0.3.10 numpy==1.17.2 scipy==1.3.1 pandas==0.25.1 scikit-learn==0.21.3 statsmodels==0.10.1 python-igraph==0.7.1 louvain==0.6.1


In [3]:
sc.settings.set_figure_params(dpi=80)

In [4]:
adata_raw = sc.read_10x_mtx('/projects/nehard/SingleCell/Aging/IL7R_HO/Aggregated/RawData/', var_names='gene_symbols', cache=True)


... reading from cache file cache/projects-nehard-SingleCell-Aging-IL7R_HO-Aggregated-RawData-matrix.h5ad


In [5]:
adata_raw.write('./IL7R_raw_090919.h5ad')

In [6]:
adata_raw =sc.read('./IL7R_raw_090919.h5ad')

In [7]:
adata_raw

AnnData object with n_obs × n_vars = 82061 × 32738 
    var: 'gene_ids'

In [9]:
#----- save meta data --> R 

adata_raw.obs.to_csv('./Meta/Meta_IL7R_raw_090919.csv')


In [10]:
#---- meta data to be added into the h5ad object 

Meta = pd.read_csv('./Meta/IL7R_MetaII_090919.csv', index_col=0)
Meta.head(4)


,Samples,IDs,Groups,Donors,Batches
AAACCTGAGATATGCA-1,1,JB17037,IL7R_Neg,H0218,B1
AAACCTGAGCGCCTTG-1,1,JB17037,IL7R_Neg,H0218,B1
AAACCTGCAAGCTGGA-1,1,JB17037,IL7R_Neg,H0218,B1
AAACCTGCAAGGACAC-1,1,JB17037,IL7R_Neg,H0218,B1


In [11]:
#---- add meta data 

adata_raw.obs['Samples']= Meta['Samples']
adata_raw.obs['IDs']= Meta['IDs']
#adata_raw.obs['Sample_ids']= Meta['Sample_ids']
adata_raw.obs['Batches']= Meta['Batches']
adata_raw.obs['Groups']= Meta['Groups']
#adata_raw.obs['Names']= Meta['Names']

adata_raw


AnnData object with n_obs × n_vars = 82061 × 32738 
    obs: 'Samples', 'IDs', 'Batches', 'Groups'
    var: 'gene_ids'

In [12]:
names = np.unique(list(adata_raw.obs['IDs']))
names

array(['JB17037', 'JB17038', 'JB17039', 'JB17040', 'JB17041', 'JB17042',
       'JB17043', 'JB17044', 'JB17045', 'JB17046', 'JB17047', 'JB17048'],
      dtype='<U7')

In [13]:
#--------------------- run Scrublet per sample

%matplotlib inline
import scrublet as scr
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import os

import numpy as np
import pandas as pd
import scanpy.api as sc

import pandas as pd


In [14]:
#--- cerate Files folder where scrubet output (1 csv files per sample)  be saved 
!mkdir Files

In [ ]:
for sample in names:
    
    cells_of_interest = adata_raw.obs.loc[adata_raw.obs["IDs"] == sample, :].index
    
    adata_i = adata_raw[cells_of_interest, :]
    
    scrub = scr.Scrublet(adata_i.X, expected_doublet_rate=0.06)
    
    doublet_scores, predicted_doublets = scrub.scrub_doublets(min_counts=2, 
                                                          min_cells=3, 
                                                          min_gene_variability_pctl=85, 
                                                          n_prin_comps=30)
    scrub.plot_histogram();
    
    print('Running UMAP...')
        
    scrub.set_embedding('UMAP', scr.get_umap(scrub.manifold_obs_, 10, min_dist=0.3))
        
    print('Done.')
        
    scrub.plot_embedding('UMAP', order_points=True);
    
    #doublet_score= pd.DataFrame({'doublet_scores' :list(doublet_scores),'predicted_doublets': list(predicted_doublets)})
        
    adata_i.obs['doublet_score']=scrub.doublet_scores_obs_
    adata_i.uns['sim_doublet_score']=scrub.doublet_scores_sim_
        
    adata_i.obs[['doublet_score']].to_csv('./Files/'+ sample + "_Scrub.csv")
    
    continue

In [17]:
import glob
import re


path = r'./Files' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.to_csv("./Files/comb_csv_IL7R.csv", index=False) 


In [19]:
Scru = pd.read_csv("./Files/comb_csv_IL7R.csv",header=0)

In [20]:
Scru = Scru.set_index('index')
Scru.head(5)

,doublet_score
index,
AAACCTGAGAGTACCG-3,0.055689
AAACCTGAGATCACGG-3,0.075000
AAACCTGAGTAGTGCG-3,0.097022
AAACCTGAGTGGTAAT-3,0.329170
AAACCTGCAATCCGAT-3,0.278588


In [21]:
adata_raw.obs['doublet_score']=Scru.doublet_score


In [22]:
adata_raw.obs.head(5)

,Samples,IDs,Batches,Groups,doublet_score
index,,,,,
AAACCTGAGATATGCA-1,1,JB17037,B1,IL7R_Neg,0.238586
AAACCTGAGCGCCTTG-1,1,JB17037,B1,IL7R_Neg,0.160714
AAACCTGCAAGCTGGA-1,1,JB17037,B1,IL7R_Neg,0.107143
AAACCTGCAAGGACAC-1,1,JB17037,B1,IL7R_Neg,0.206494
AAACCTGCACGGCTAC-1,1,JB17037,B1,IL7R_Neg,0.117551


In [23]:
adata_raw

AnnData object with n_obs × n_vars = 82061 × 32738 
    obs: 'Samples', 'IDs', 'Batches', 'Groups', 'doublet_score'
    var: 'gene_ids'

In [24]:
#--- Define threshold to be applyed + define doublet vs. singlet
threshold = 0.18
adata_raw.obs['predicted_doublets_1'] = pd.Categorical(['doublet' if x > threshold else 'singlet' for x in adata_raw.obs.doublet_score])

In [25]:
adata_raw.obs.predicted_doublets_1.value_counts()

singlet    63211
doublet    18850
Name: predicted_doublets_1, dtype: int64

In [27]:
cells_of_interest1 = adata_raw.obs.loc[adata_raw.obs["predicted_doublets_1"].isin(["singlet"]), :].index
cells_of_interest1

Index(['AAACCTGAGCGCCTTG-1', 'AAACCTGCAAGCTGGA-1', 'AAACCTGCACGGCTAC-1',
       'AAACCTGCAGACTCGC-1', 'AAACCTGCATCGTCGG-1', 'AAACCTGGTGAGGCTA-1',
       'AAACCTGGTTACAGAA-1', 'AAACCTGGTTACGACT-1', 'AAACCTGTCATCGGAT-1',
       'AAACCTGTCGGTTAAC-1',
       ...
       'TTTGGTTTCTTCGGTC-12', 'TTTGTCAAGCTCAACT-12', 'TTTGTCAAGTGCTGCC-12',
       'TTTGTCACAAACGTGG-12', 'TTTGTCACATAGGATA-12', 'TTTGTCAGTAGTGAAT-12',
       'TTTGTCAGTCATCCCT-12', 'TTTGTCAGTGCCTTGG-12', 'TTTGTCATCAGCGACC-12',
       'TTTGTCATCGTCCAGG-12'],
      dtype='object', name='index', length=63211)

In [28]:
adata_Scrub=adata_raw[cells_of_interest1, :]
adata_Scrub

View of AnnData object with n_obs × n_vars = 63211 × 32738 
    obs: 'Samples', 'IDs', 'Batches', 'Groups', 'doublet_score', 'predicted_doublets_1'
    var: 'gene_ids'

In [30]:
#-- save the onject after doublet removal 

adata_Scrub.write('./IL7R_Scrub_BC_090919.h5ad')

Trying to set attribute `.obs` of view, making a copy.
... storing 'IDs' as categorical
Trying to set attribute `.obs` of view, making a copy.
... storing 'Batches' as categorical
Trying to set attribute `.obs` of view, making a copy.
... storing 'Groups' as categorical
